In [1]:
import pandas as pd
import numpy as np

# Ejercicio 1

In [2]:
data = pd.read_excel('18.- Repaso_2_EX_P2025.xlsx', sheet_name=1)
data = data.set_index('Date')
data.head()

,Bid,Ask,Volume,Bid.1,Ask.1,Volume.1,Bid.2,Ask.2,Volume.2
Date,,,,,,,,,
2024-02-28,2039.95,2040.60,76761.0,22.665,22.675,46918.0,3.8390,3.8395,56401.0
2024-02-27,2040.94,2044.34,213033.0,22.400,22.475,43137.0,3.8530,3.8540,55029.0
2024-02-26,2034.10,2035.99,137431.0,22.515,22.530,38360.0,3.8350,3.8370,66368.0
2024-02-23,2033.40,2038.00,139360.0,22.975,22.980,53064.0,3.8685,3.8695,54672.0
2024-02-22,2026.26,2032.28,78097.0,22.775,22.780,61639.0,3.8945,3.8950,64774.0


In [3]:
def VaR_APL(data: pd.DataFrame, posiciones: np.array, conf: float, long: bool):
    """ 
    Función que calcula el VaR y C-VaR de un portafolio de activos financieros ajustado por costos de liquidez.

    Args:
    data (pd.DataFrame): DataFrame con los bid y ask de los activos financieros.
    posiciones (np.array): Vector con las posiciones de los activos financieros, en el orden de 'data'.
    conf (float): Nivel de confianza con rango [0,1].
    long (bool): True si el portafolio es largo, False si es corto.

    Returns:
    pd.DataFrame: DataFrame con el VaR y C-VaR ajustado por costos de liquidez.
    """

    data = data.sort_index()

    # Bid y Ask
    bid_columns = [col for col in data.columns if 'Bid' in col] # Selecciona las columnas que contienen 'Bid'
    ask_columns = [col for col in data.columns if 'Ask' in col] # Selecciona las columnas que contienen 'Ask'

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Mid
    data[mid_columns] = (data[bid_columns].values + data[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Spread
    data[spread_columns] = (data[ask_columns].values - data[bid_columns].values) / data[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))] # Se crea una lista con los nombres de las columnas de Return
    data[return_columns] = data[mid_columns].pct_change()

    # Weights
    value = posiciones * data[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    data['port_ret'] = np.dot(data[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(data['port_ret'].dropna(), 100 - conf*100) if long else np.percentile(data['port_ret'].dropna(), conf*100)
    var_cash = pv * var_pct

    # C-VaR calculation
    cvar_pct = data['port_ret'][data['port_ret'] < var_pct].dropna().mean() if long else data['port_ret'][data['port_ret'] > var_pct].dropna().mean()
    cvar_cash = pv * cvar_pct

    # Liquidity cost
    cl_prom = data[spread_columns].mean()
    cl_estr = np.percentile(data[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost
    var_apl_prom, var_apl_estr = np.abs(((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr))))

    var_apl_prom_cash, var_apl_estr_cash = np.abs(((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr))))
    
    # C-VaR adjusted by liquidity cost
    cvar_apl_prom, cvar_apl_estr = np.abs(((cvar_pct - np.dot(w, cl_prom), cvar_pct - np.dot(w, cl_estr)) if long
                                    else (cvar_pct + np.dot(w, cl_prom), cvar_pct + np.dot(w, cl_estr))))
    
    cvar_apl_prom_cash, cvar_apl_estr_cash = np.abs(((cvar_cash - np.dot(value, cl_prom), cvar_cash - np.dot(value, cl_estr)) if long
                                            else (cvar_cash + np.dot(value, cl_prom), cvar_cash + np.dot(value, cl_estr))))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado', 'C-VaR', 'C-VaR Ajustado Promedio', 'C-VaR Ajustado Estresado'],
        'Porcentaje': [np.abs(var_pct), var_apl_prom, var_apl_estr, np.abs(cvar_pct), cvar_apl_prom, cvar_apl_estr],
        'Cash': [np.abs(var_cash), var_apl_prom_cash, var_apl_estr_cash, np.abs(cvar_cash), cvar_apl_prom_cash, cvar_apl_estr_cash]
    })

    return resultados

In [4]:
posiciones = np.array([121*100, 153*5000, 189*25000])
conf = 0.99
long = False

VaR_APL(data, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.024430,1.469943e+06
1,VaR Ajustado Promedio,0.025413,1.529102e+06
2,VaR Ajustado Estresado,0.028933,1.740885e+06
3,C-VaR,0.028891,1.738383e+06
4,C-VaR Ajustado Promedio,0.029874,1.797542e+06
5,C-VaR Ajustado Estresado,0.033394,2.009325e+06


## Ndays

In [5]:
posicion = np.array([121*100])
ADV = data["Volume"].head(89).mean()

Ndays = posicion / ADV
np.ceil(Ndays)

array([1.])

# Ejercicio 2

In [7]:
data = pd.read_excel('18.- Repaso_2_EX_P2025.xlsx', sheet_name=3)
data = data.set_index('Date')
posiciones = np.array([1_571_000, 7_203_000])
conf = 0.99
long = True

VaR_APL(data, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.014700,46302.291826
1,VaR Ajustado Promedio,0.014965,47136.154157
2,VaR Ajustado Estresado,0.015261,48068.543699
3,C-VaR,0.017098,53853.467223
4,C-VaR Ajustado Promedio,0.017362,54687.329554
5,C-VaR Ajustado Estresado,0.017658,55619.719097


In [8]:
def VaR_APL(data: pd.DataFrame, posiciones: np.array, conf: float, long: bool):
    """ 
    Función que calcula el VaR y C-VaR de un portafolio de activos financieros ajustado por costos de liquidez.

    Args:
    data (pd.DataFrame): DataFrame con los bid y ask de los activos financieros.
    posiciones (np.array): Vector con las posiciones de los activos financieros, en el orden de 'data'.
    conf (float): Nivel de confianza con rango [0,1].
    long (bool): True si el portafolio es largo, False si es corto.

    Returns:
    pd.DataFrame: DataFrame con el VaR y C-VaR ajustado por costos de liquidez.
    """

    data = data.sort_index()

    # Bid y Ask
    bid_columns = [col for col in data.columns if 'Bid' in col] # Selecciona las columnas que contienen 'Bid'
    ask_columns = [col for col in data.columns if 'Ask' in col] # Selecciona las columnas que contienen 'Ask'

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Mid
    data[mid_columns] = (data[bid_columns].values + data[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Spread
    data[spread_columns] = (data[ask_columns].values - data[bid_columns].values) / data[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))] # Se crea una lista con los nombres de las columnas de Return
    data[return_columns] = data[mid_columns].pct_change()

    # Weights
    value = posiciones * data[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    data['port_ret'] = np.dot(data[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(data['port_ret'].dropna(), 100 - conf*100) if long else np.percentile(data['port_ret'].dropna(), conf*100)
    var_cash = pv * var_pct

    # C-VaR calculation
    cvar_pct = data['port_ret'][data['port_ret'] < var_pct].dropna().mean() if long else data['port_ret'][data['port_ret'] > var_pct].dropna().mean()
    cvar_cash = pv * cvar_pct

    # Liquidity cost
    cl_prom = data[spread_columns].mean()
    cl_estr = np.percentile(data[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost
    var_apl_prom, var_apl_estr = np.abs(((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr))))

    var_apl_prom_cash, var_apl_estr_cash = np.abs(((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr))))
    
    # C-VaR adjusted by liquidity cost
    cvar_apl_prom, cvar_apl_estr = np.abs(((cvar_pct - np.dot(w, cl_prom), cvar_pct - np.dot(w, cl_estr)) if long
                                    else (cvar_pct + np.dot(w, cl_prom), cvar_pct + np.dot(w, cl_estr))))
    
    cvar_apl_prom_cash, cvar_apl_estr_cash = np.abs(((cvar_cash - np.dot(value, cl_prom), cvar_cash - np.dot(value, cl_estr)) if long
                                            else (cvar_cash + np.dot(value, cl_prom), cvar_cash + np.dot(value, cl_estr))))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado', 'C-VaR', 'C-VaR Ajustado Promedio', 'C-VaR Ajustado Estresado'],
        'Porcentaje': [np.abs(var_pct), var_apl_prom, var_apl_estr, np.abs(cvar_pct), cvar_apl_prom, cvar_apl_estr],
        'Cash': [np.abs(var_cash), var_apl_prom_cash, var_apl_estr_cash, np.abs(cvar_cash), cvar_apl_prom_cash, cvar_apl_estr_cash]
    })

    return np.dot(value, cl_prom) #Cambiar dependiendo de lo que se pide

In [9]:
VaR_APL(data, posiciones, conf, long)

833.8623314070503